In [1]:
#Import Libraries
import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
# TfidfVectorizer - This is used to convert text data into numerical values
from sklearn.metrics.pairwise import cosine_similarity
import os

In [2]:
os.getcwd()

'C:\\Users\\suman'

In [4]:
os.chdir('C:\\Users\\suman\\OneDrive\\Desktop\\Sunny\\Acmegrade machine learning\\Dec 23 ML Day 17\\Dec 23 ML Day 17\\Projects\\PRJ Movie Recommendation System')

In [5]:
os.getcwd()

'C:\\Users\\suman\\OneDrive\\Desktop\\Sunny\\Acmegrade machine learning\\Dec 23 ML Day 17\\Dec 23 ML Day 17\\Projects\\PRJ Movie Recommendation System'

In [7]:
movies_data =pd.read_csv('movies.csv')
movies_data.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [8]:
#Display the shape
#shape means number of rows and columns
display (movies_data.shape)

(4803, 24)

In [9]:
#Selecting the relevant features for recommendation
selected_features = ['genres','keywords','tagline','cast','director']
print(selected_features)

['genres', 'keywords', 'tagline', 'cast', 'director']


In [10]:
#Display the info
display (movies_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   index                 4803 non-null   int64  
 1   budget                4803 non-null   int64  
 2   genres                4775 non-null   object 
 3   homepage              1712 non-null   object 
 4   id                    4803 non-null   int64  
 5   keywords              4391 non-null   object 
 6   original_language     4803 non-null   object 
 7   original_title        4803 non-null   object 
 8   overview              4800 non-null   object 
 9   popularity            4803 non-null   float64
 10  production_companies  4803 non-null   object 
 11  production_countries  4803 non-null   object 
 12  release_date          4802 non-null   object 
 13  revenue               4803 non-null   int64  
 14  runtime               4801 non-null   float64
 15  spoken_languages     

None

In [11]:
#Check for Null Values
display (movies_data.isna().sum())

index                      0
budget                     0
genres                    28
homepage                3091
id                         0
keywords                 412
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
cast                      43
crew                       0
director                  30
dtype: int64

In [12]:
#Display the selected columns
display (movies_data[selected_features].head())

,genres,keywords,tagline,cast,director
0,Action Adventure Fantasy Science Fiction,culture clash future space war space colony so...,Enter the World of Pandora.,Sam Worthington Zoe Saldana Sigourney Weaver S...,James Cameron
1,Adventure Fantasy Action,ocean drug abuse exotic island east india trad...,"At the end of the world, the adventure begins.",Johnny Depp Orlando Bloom Keira Knightley Stel...,Gore Verbinski
2,Action Adventure Crime,spy based on novel secret agent sequel mi6,A Plan No One Escapes,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,Sam Mendes
3,Action Crime Drama Thriller,dc comics crime fighter terrorist secret ident...,The Legend Ends,Christian Bale Michael Caine Gary Oldman Anne ...,Christopher Nolan
4,Action Adventure Science Fiction,based on novel mars medallion space travel pri...,"Lost in our world, found in another.",Taylor Kitsch Lynn Collins Samantha Morton Wil...,Andrew Stanton


In [13]:
#Check Null Values in selected columns
display (movies_data[selected_features].isna().sum())

genres       28
keywords    412
tagline     844
cast         43
director     30
dtype: int64

In [17]:
selected_features

['genres', 'keywords', 'tagline', 'cast', 'director']

In [15]:
#Replacing the null values with null string
#there are many null values in above result so to remove the null values we are replacing it by spaces
for feature in selected_features:
    movies_data[feature] = movies_data[feature].fillna('')
display (movies_data.head())

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [18]:
#Check Null Values in selected columns
display (movies_data[selected_features].isna().sum())

genres      0
keywords    0
tagline     0
cast        0
director    0
dtype: int64

In [20]:
#Combining all the 5 selected features
combined_features = movies_data['genres']+' '+movies_data['keywords']+''+movies_data['tagline']+' '+movies_data['cast']+' '+movies_data['director']
display (combined_features)

0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
4798    Action Crime Thriller united states\u2013mexic...
4799    Comedy Romance A newlywed couple's honeymoon i...
4800    Comedy Drama Romance TV Movie date love at fir...
4801     A New Yorker in Shanghai Daniel Henney Eliza ...
4802    Documentary obsession camcorder crush dream gi...
Length: 4803, dtype: object

In [21]:
#Converting the text data to feature vectors
vectorizer = TfidfVectorizer()
feature_vectors = vectorizer.fit_transform(combined_features)
display (feature_vectors.shape)
print (feature_vectors)

(4803, 20291)

  (0, 2825)	0.17365733752396292
  (0, 9224)	0.11349377091205677
  (0, 15280)	0.19528566108954704
  (0, 11950)	0.16145450217509352
  (0, 10299)	0.22831712302293364
  (0, 17141)	0.152325312366219
  (0, 19540)	0.20065059297287455
  (0, 16470)	0.20707370758386553
  (0, 15611)	0.2189235377509099
  (0, 20254)	0.2030704154867694
  (0, 19951)	0.2377107082949574
  (0, 15641)	0.15102423741048945
  (0, 13458)	0.2735833232498806
  (0, 13114)	0.09102271907804357
  (0, 19935)	0.1296922122527455
  (0, 17938)	0.07835679145512903
  (0, 16721)	0.28682283809133036
  (0, 3741)	0.2569035071769314
  (0, 19434)	0.12763467115712607
  (0, 16843)	0.3463021691447409
  (0, 7007)	0.16761913103948795
  (0, 3561)	0.22831712302293364
  (0, 4316)	0.21507760818148391
  (0, 6451)	0.1042622339194933
  (0, 15922)	0.10438430008517753
  :	:
  (4801, 20230)	0.288447054553863
  (4801, 5749)	0.24699827490898943
  (4801, 520)	0.1771758756927387
  (4801, 8230)	0.288447054553863
  (4801, 13645)	0.21545343228494443
  (4801, 1967)	

In [22]:
#Getting the similarity scores using cosine similarity
similarity = cosine_similarity(feature_vectors)
print (similarity )

[[1.         0.07376915 0.03750476 ... 0.         0.         0.        ]
 [0.07376915 1.         0.03276527 ... 0.01216731 0.         0.        ]
 [0.03750476 0.03276527 1.         ... 0.         0.05322545 0.        ]
 ...
 [0.         0.01216731 0.         ... 1.         0.         0.02626214]
 [0.         0.         0.05322545 ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.02626214 0.         1.        ]]


In [23]:
display (similarity.shape )

(4803, 4803)

In [26]:
pd.DataFrame(similarity).to_csv ('cosine similarity.csv')

In [27]:
#Enter the movie name to get Similarity
movie_name = input(' Enter your favourite movie name : ')

 Enter your favourite movie name :  pirates of the caribean


In [28]:
#Creating a list with all the movie names given in the dataset
list_of_all_titles = movies_data['title'].tolist()
print(list_of_all_titles)

['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron', 'Harry Potter and the Half-Blood Prince', 'Batman v Superman: Dawn of Justice', 'Superman Returns', 'Quantum of Solace', "Pirates of the Caribbean: Dead Man's Chest", 'The Lone Ranger', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Avengers', 'Pirates of the Caribbean: On Stranger Tides', 'Men in Black 3', 'The Hobbit: The Battle of the Five Armies', 'The Amazing Spider-Man', 'Robin Hood', 'The Hobbit: The Desolation of Smaug', 'The Golden Compass', 'King Kong', 'Titanic', 'Captain America: Civil War', 'Battleship', 'Jurassic World', 'Skyfall', 'Spider-Man 2', 'Iron Man 3', 'Alice in Wonderland', 'X-Men: The Last Stand', 'Monsters University', 'Transformers: Revenge of the Fallen', 'Transformers: Age of Extinction', 'Oz: The Great and Powerful', 'The Amazing Spider-Man 2', 'TRON: Legacy', 'Cars 2', 'Green Lant

In [29]:
#Length of the List
len(list_of_all_titles)

4803

In [30]:
#Finding the close match for the movie name given by the user
find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)
print(find_close_match)

['Rise of the Guardians', 'Wrath of the Titans', "Pirates of the Caribbean: At World's End"]


In [31]:
#Display the close match
close_match = find_close_match[0]
print(close_match)

Rise of the Guardians


In [32]:
#Finding the index of the movie with the title
index_of_the_movie = movies_data[movies_data.title ==
close_match]['index'].values[0]
print(index_of_the_movie)

154


In [33]:
#Get the similarity row for the selected index
similarity_score = list(enumerate(similarity[index_of_the_movie]))
print(similarity_score)

[(0, 0.012138807462217845), (1, 0.01028897972234677), (2, 0.0), (3, 0.0), (4, 0.0), (5, 0.012354758460828657), (6, 0.025763866513424008), (7, 0.03485883407313167), (8, 0.022400738271916718), (9, 0.012030229181295779), (10, 0.013456442385202933), (11, 0.0), (12, 0.012151414485262926), (13, 0.0), (14, 0.048146819112059105), (15, 0.02050773858414094), (16, 0.03329081870481547), (17, 0.011802764833709121), (18, 0.0), (19, 0.025424816418695595), (20, 0.011744455729350027), (21, 0.0), (22, 0.027470619510233406), (23, 0.02552297554012104), (24, 0.016148574610841307), (25, 0.04031623780996372), (26, 0.018529936499398547), (27, 0.015588729050575083), (28, 0.01717797939407318), (29, 0.0), (30, 0.011920968889778338), (31, 0.0), (32, 0.03110291990015697), (33, 0.06938610076142127), (34, 0.042558216453853576), (35, 0.0), (36, 0.0), (37, 0.022447272836341278), (38, 0.01234827385614303), (39, 0.0), (40, 0.02761056790647013), (41, 0.016493146974117526), (42, 0.025766383438139882), (43, 0.0), (44, 0.0)

In [34]:
#Length of Similarity Score
len(similarity_score)

4803

In [35]:
#Sorting the movies based on their similarity score
sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse =
True)
print(sorted_similar_movies)

[(154, 1.0), (2019, 0.2051145545696792), (686, 0.19542975573156593), (90, 0.18846415736246466), (406, 0.14380702334123308), (2292, 0.14347479280750622), (211, 0.1239338567338073), (178, 0.11750464601448299), (636, 0.11719116550486075), (1494, 0.11504612689569478), (143, 0.11160705925781747), (2540, 0.10958776960253958), (3768, 0.10880331726896364), (727, 0.10600626524343926), (637, 0.10560258598187511), (142, 0.10409382737600946), (504, 0.1021030862645355), (1587, 0.09985592187229307), (1230, 0.09952491822469861), (1910, 0.09765502241763892), (3670, 0.09619636215232175), (3345, 0.0960523226158491), (168, 0.0935288163073709), (1658, 0.09280696342954517), (3949, 0.09193381157330366), (1288, 0.089546534058084), (1408, 0.08856739064310967), (3589, 0.08796017128022789), (265, 0.08702636562314849), (519, 0.08689892252902734), (385, 0.08418382346439615), (2931, 0.08414013421988598), (917, 0.08182722116091079), (481, 0.08160697078132453), (1424, 0.08132858963322837), (762, 0.0808830729724459),

In [37]:
#Print the name of similar movies based on the index – Top 30
print('Movies suggested for you : \n')
i = 1
for movie in sorted_similar_movies:
    index = movie[0]
    title_from_index = movies_data[movies_data.index==index]['title'].values[0]
    if (i<30):
        print(i, '.',title_from_index)
        i+=1

Movies suggested for you : 

1 . Rise of the Guardians
2 . The Santa Clause
3 . Hop
4 . The Polar Express
5 . Arthur Christmas
6 . Thomas and the Magic Railroad
7 . How the Grinch Stole Christmas
8 . Rango
9 . The Santa Clause 2
10 . Elf
11 . Pan
12 . Krampus
13 . The Real Cancun
14 . Jingle All the Way
15 . Les Misérables
16 . Flushed Away
17 . The Secret Life of Pets
18 . The Curse of the Were-Rabbit
19 . The Shadow
20 . Hot Rod
21 . Running Forever
22 . Definitely, Maybe
23 . Final Fantasy: The Spirits Within
24 . The Imaginarium of Doctor Parnassus
25 . Bachelorette
26 . The Last Shot
27 . Closer
28 . Still Alice
29 . The Cat in the Hat


In [39]:
#Consolidated code
movie_name = input(' Enter your favourite movie name : ')
list_of_all_titles = movies_data['title'].tolist()
find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)
close_match = find_close_match[0]
index_of_the_movie = movies_data[movies_data.title ==
close_match]['index'].values[0]
similarity_score = list(enumerate(similarity[index_of_the_movie]))
sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse =
True)
print('Movies suggested for you : \n')
i = 1
for movie in sorted_similar_movies:
    index = movie[0]
    title_from_index = movies_data[movies_data.index==index]['title'].values[0]
    if (i<30):
        print(i, '.',title_from_index)
        i+=1

 Enter your favourite movie name :  dead pool


Movies suggested for you : 

1 . Deadpool
2 . Iron Man 2
3 . X-Men
4 . The Incredible Hulk
5 . Avengers: Age of Ultron
6 . X-Men: Apocalypse
7 . Thor: The Dark World
8 . The Avengers
9 . Ant-Man
10 . X2
11 . The Amazing Spider-Man 2
12 . X-Men: The Last Stand
13 . X-Men: Days of Future Past
14 . Serenity
15 . Haywire
16 . The Image Revolution
17 . Spider-Man 2
18 . Spawn
19 . The Spirit
20 . Superman II
21 . Kick-Ass 2
22 . Man of Steel
23 . Thor
24 . Superhero Movie
25 . Batman v Superman: Dawn of Justice
26 . The Peanuts Movie
27 . X-Men: First Class
28 . Big Hero 6
29 . Captain America: The Winter Soldier
